# ETL Pipeline:

Creating a preprocessing pipleine to extract the data, transform the data according to our solution needs and load the data into a data base to be used later.   

In [1]:
import numpy as np
import pandas as pd
import itertools
from collections import Counter
from nltk.corpus import stopwords

In [3]:
class ETL():
    
    #initializing the variables
    def __init__(self,Amazon_filepath,Filepath):

        #assigning csv file to Netflix_file variable 
        self.Amazon_filepath = Amazon_filepath

        #assigning a address to save the cleaned data
        self.Filepath = Filepath

        #downloading Netflix datset
        self.data = pd.read_json(self.Amazon_filepath)

            
    #Data loading
    def load_data(self):

        #printing the shape of our data set 
        print('The data has {} data points and {} features \n'.format(self.data.shape[0],self.data.shape[1]))
        #printing a space for presentation

        #keeping just the pertinent features
        self.data = self.data[['asin','product_type_name', 'formatted_price','title','medium_image_url']]

        print('The data after removing irrelevant features has {} and it contains these {} features. The names of the features are {} \n \n'
              .format(self.data.shape[0],self.data.shape[1],list(self.data.columns)))
    
    #Data analysis
    def analysis(self):

        #Basic stats for product type
        print('The basic statistics for product type on amazon are as follows: \n{}\n\n'
              .format(self.data['product_type_name'].describe()))

        #product type segregation
        print('Product type count:\n{}\n\n'.format(Counter(list(self.data['product_type_name'])).most_common(10)))

        #basic stats for titles
        print('The basic statistics for product type on amazon are as follows: \n{}\n\n'.format(self.data['title'].describe()))

        #Basic stats for product type
        print('{} % of the total points have a listed price \n \n'
              .format(self.data[~self.data['formatted_price'].isnull()].shape[0]/self.data.shape[0]*100))

    #Data cleaning
    def Data_cleaning(self):
        
        # removing products without a price as we need a price to sell products
        self.data = self.data[~self.data['formatted_price'].isnull()]
        print('The number of products (data points) remaining after removing products without a price: \n{}\n'
              .format(self.data.shape[0]))

        
        #removing products without a title as we need titles for vectorization
        #distance based similarity recommendation for title vectorization
        self.data = self.data[~self.data['title'].isnull()]
        print('The number of products (data points) remaining after removing products without a title description required for vectorization:\n{}\n'
              .format(self.data.shape[0]))

        
        #removing products with small length titles as they might not adequately describe product
        self.data = self.data[self.data['title'].apply(lambda x : len(x.split())>4)]
        print('The number of products (data points) remaining after removing products with insufficient title descriptions required for vectorization:\n{}\n'
              .format(self.data.shape[0]))
         
        #removing duplicate 'titles'
        #Below is the code to remove similar titles with just 3 words differing from its duplicate 
        #################################################### start ###########################################################
        indices = []
        for i,row in self.data.iterrows():
            indices.append(i)
        
        asins = []
        while len(indices)!= 0:
            i = indices.pop()

            asins.append(self.data['asin'].loc[i])

            a = self.data['title'].loc[i].split()
        
        # store the list of words of ith string in a lista = data['title'].loc[i].spilt()
        for j in indices:
            
            # store the list of words of jth string in a list b = data['title'].loc[j].spilt()
            b = self.data['title'].loc[j].split()
            
            #storing the max len between list a or b
            length = max(len(a),len(b))
            
            # count is used to store the number of words that are matched in both lists
            count = 0
            
            # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strng
            # example: a =['a', 'b', 'c', 'd']
            # b = ['a', 'b', 'd']
            # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
            for h in itertools.zip_longest(a,b):
                if (h[0]==h[1]):
                    count += 1                    
                
            if (length - count) < 3:
                indices.remove(j)            

        #keeping product data points without a duplicate
        self.data = self.data[self.data['asin'].isin(asins)]

        print('The number of products (data points) remaining after removing products with duplicate title descriptions:\n{}\n\n'
              .format(self.data.shape[0]))
        ####################################################################################################################            
        
        
        
        #removing stopwords, terms which are not alphanumeric and lowering text
        stopword = set(stopwords.words('english'))
        
        for index,rows in self.data.iterrows():
            strng = ""
            for words in rows['title'].split():

                #removing special characters 
                word = ("".join(i for i in words if i.isalnum()))

                #lowering the words
                word = word.lower()
                
                #removing stopwords
                if word not in stopword:
                    strng += word + " "         
            self.data['title'].loc[index] = strng
            
    def save_data(self):
    
        #saving data in a pickle file
        self.data.to_pickle(self.Filepath)
        
    def processed(self):
    
        print('LOADING DATA...{}\n\n '.format(self.Amazon_filepath))
        self.load_data()

        print('DATA ANALYSIS...\n\n')
        self.analysis()
        
        print('CLEANING DATA...\n\n')
        self.Data_cleaning()
        
        print('SAVING DATA IN PICKLE FILE PREPROCESSED {}...\n\n'.format(self.Filepath))
        self.save_data()
        
        print('Cleaned data saved to pickle file preprocessed in Pickle folder')